In [60]:
import sqlite3
import pandas as pd
from pathlib import Path

import os
import math
import random
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import networkx as nx

In [61]:
# path to databases
"For which experiment would you like to add the density matrix to the database?"
experiment = int(input())
if experiment == 1:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp1.db')
    #db_path = Path('E:/HumanA/Data/HumanA_Exp1_WorkingData.db')
elif experiment == 2:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp2.db')
    #db_path = Path('E:/HumanA/Data/HumanA_Exp2_WorkingData.db')

2


In [62]:
# check if path exists
if not db_path or not db_path.exists():
    db_path = ':memory:'

In [27]:
def create_density_matrix(coordinates, meters):
    xmin = coordinates["x"].min()
    xmax = coordinates["x"].max()
    zmin = coordinates["z"].min()
    zmax = coordinates["z"].max()
    step = meters
    xsize = int((np.abs(xmin) + np.abs(xmax)) / step) + 1
    zsize = int((np.abs(zmin) + np.abs(zmax)) / step) + 1
    density_matrix = np.zeros((xsize,zsize), dtype=np.int64)
    i = 0
    for coordinate in coordinates[["x", "z"]].to_numpy():
        x = int((np.abs(xmin) + coordinate[0]) / step)
        y = int((np.abs(zmin) + coordinate[1]) / step)
        density_matrix[x,y] += 1
        i += 1
    return density_matrix

In [28]:
def convert_coordinate(coordinate, meters):
    step = meters
    xsize = int((np.abs(xmin) + np.abs(xmax)) / step) + 1
    zsize = int((np.abs(zmin) + np.abs(zmax)) / step) + 1
    x = int((np.abs(xmin) + coordinate[0]) / step)
    y = int((np.abs(zmin) + coordinate[1]) / step)
    return x, y

In [29]:
def rotate_matrix(matrix):
    return np.rot90(np.flip(matrix, axis=0), k=-1)

In [30]:
def create_matrix(step, xmin, xmax, zmin,zmax):
    xsize = int((np.abs(xmin) + np.abs(xmax)) / step) + 1
    zsize = int((np.abs(zmin) + np.abs(zmax)) / step) + 1
    density_matrix = np.zeros((xsize,zsize), dtype=np.int64)
    return density_matrix

In [31]:
def create_trial_matrix(coordinates, meters, xmin, xmax, zmin, zmax):
    step = meters
    trial_matrix = create_matrix(step = step, xmin = xmin, xmax = xmax, zmin = zmin, zmax = zmax)
    #print(trial_matrix.shape)
    i = 0
    for coordinate in coordinates[["x", "z"]].to_numpy():
        x = int((np.abs(xmin) + coordinate[0]) / step)
        y = int((np.abs(zmin) + coordinate[1]) / step)
        trial_matrix[x,y] += 1
        i += 1
    return trial_matrix

In [65]:
# connect to database
connection=sqlite3.connect(db_path)
cr=connection.cursor()

# read data from SQL to pandas dataframe.
sql_instruction = """
SELECT DISTINCT participantId FROM trials WHERE validParticipant = 'VALID';
"""
cr.execute(sql_instruction)
participants = cr.fetchall()

In [33]:
# create a table that stores the mask information
sql_instruction = """
CREATE TABLE IF NOT EXISTS "density_map_participants"(
"Id" INTEGER NOT NULL UNIQUE,
"TrialId" INTEGER NOT NULL,
"Row" NUMERIC,
"Column" NUMERIC,
"Value" NUMERIC,
PRIMARY KEY ("Id" AUTOINCREMENT)
FOREIGN KEY(TrialId) REFERENCES trials(Id)
);
"""
cr.execute(sql_instruction)

In [34]:
connection.commit()

In [35]:
sql_instruction = """
SELECT MIN(playerBodyPosition_x) FROM data_points;
"""
cr.execute(sql_instruction)

xMin = cr.fetchone()
xMin = round(xMin[0],2)
print(xMin)

-442.87


In [36]:
sql_instruction = """
SELECT MAX(playerBodyPosition_x) FROM data_points;
"""
cr.execute(sql_instruction)

xMax = cr.fetchone()
xMax = round(xMax[0],2)
print(xMax)

439.76


In [37]:
sql_instruction = """
SELECT MIN(playerBodyPosition_z) FROM data_points;
"""
cr.execute(sql_instruction)

zMin = cr.fetchone()
zMin = round(zMin[0],2)
print(zMin)

-293.49


In [38]:
sql_instruction = """
SELECT MAX(playerBodyPosition_z) FROM data_points;
"""
cr.execute(sql_instruction)

zMax = cr.fetchone()
zMax = round(zMax[0],2)
print(zMax)

305.67


In [39]:
# connect to database
#connection=sqlite3.connect(db_path_exp1)
#cr=connection.cursor()
density_matrix = create_matrix(step = 4, xmin = xMin, xmax = xMax, zmin = zMin, zmax = zMax)
for participant in participants:
    participant = participant[0]
    
    sql_instruction = f"""
    SELECT DISTINCT id 
    FROM trials
    WHERE participantId = {participant};
    """
    cr.execute(sql_instruction)
    trialId = cr.fetchall()
    #print(trialId_exp1)
    #trialId_exp1 = trialId_exp1[0]
    #print(trialId_exp1)
    for trialId in trialId:
        trialId = trialId[0]
        
        sql_instruction = """SELECT DISTINCT TrialId FROM density_map_participants"""
        cr.execute(sql_instruction)
        trial_idx = cr.fetchall()
        if (trialId,) not in trial_idx:
            print("Participant: " + str(participant) + " | " + "TrialId: " + str(trialId))

            # get relevant data (participant ID, sessionNr, timestamp, x,y,z coordinates) from database for current trial
            sql_instruction = f"""
            SELECT trials.participantId, trials.sessionNr, data_points.timeStampDataPointStart, 
                data_points.playerBodyPosition_x, data_points.playerBodyPosition_y, 
                data_points.playerBodyPosition_z
            FROM trials
            LEFT JOIN data_points ON trials.id = data_points.trialId
            WHERE trials.id = {trialId}
            ;
            """

            tracking_data = pd.read_sql_query(sql_instruction, connection)
            
            # rename columns 
            tracking_data.columns = ['participant', 'session', 'time', 'x', 'y', 'z']
            
            # round coordinates to 2 digits
            tracking_data["x"] = tracking_data["x"].round(2)
            tracking_data["z"] = tracking_data["z"].round(2)
            

            # create density matrix for trial
            density_matrix_trial = create_trial_matrix(coordinates=tracking_data, meters=4, xmin = xMin, 
                                                       xmax = xMax, zmin = zMin, zmax = zMax)
        
            # get density matrix values & store in database (every value that is not 0)
            for row in range(density_matrix_trial.shape[0]):
                for column in range(density_matrix_trial.shape[1]):
                    if density_matrix_trial[row][column] != 0:
                        value = str((trialId,row,column, density_matrix_trial[row][column]))
                        sql_instruction = f"""INSERT INTO density_map_participants (TrialId, Row, Column, Value) VALUES {value};"""
                        cr.execute(sql_instruction)
            connection.commit()

    #print("Do you want to continue (next participant) ?")
    #_continue = input()
    #if _continue == "Yes":
    #    continue
    #else:
    #    break
print("done")
#connection.close()


Participant: 1754 | TrialId: 7
Participant: 1754 | TrialId: 9
Participant: 1754 | TrialId: 11
Participant: 1754 | TrialId: 13
Participant: 1754 | TrialId: 15
Participant: 1754 | TrialId: 18
Participant: 1754 | TrialId: 21
Participant: 1754 | TrialId: 23
Participant: 1754 | TrialId: 25
Participant: 1754 | TrialId: 27
Participant: 1754 | TrialId: 29
Participant: 1754 | TrialId: 32
Participant: 1754 | TrialId: 36
Participant: 1754 | TrialId: 40
Participant: 1754 | TrialId: 44
Participant: 2258 | TrialId: 16
Participant: 2258 | TrialId: 19
Participant: 2258 | TrialId: 22
Participant: 2258 | TrialId: 24
Participant: 2258 | TrialId: 26
Participant: 2258 | TrialId: 28
Participant: 2258 | TrialId: 31
Participant: 2258 | TrialId: 34
Participant: 2258 | TrialId: 37
Participant: 2258 | TrialId: 41
Participant: 2258 | TrialId: 45
Participant: 2258 | TrialId: 49
Participant: 2258 | TrialId: 53
Participant: 2258 | TrialId: 57
Participant: 2258 | TrialId: 62
Participant: 2693 | TrialId: 38
Participan

Participant: 9472 | TrialId: 303
Participant: 9472 | TrialId: 311
Participant: 9472 | TrialId: 321
Participant: 9472 | TrialId: 330
Participant: 9472 | TrialId: 340
Participant: 9472 | TrialId: 347
Participant: 9472 | TrialId: 353
Participant: 9472 | TrialId: 359
Participant: 9472 | TrialId: 361
Participant: 9472 | TrialId: 365
Participant: 9472 | TrialId: 367
Participant: 9601 | TrialId: 276
Participant: 9601 | TrialId: 286
Participant: 9601 | TrialId: 297
Participant: 9601 | TrialId: 308
Participant: 9601 | TrialId: 317
Participant: 9601 | TrialId: 326
Participant: 9601 | TrialId: 334
Participant: 9601 | TrialId: 343
Participant: 9601 | TrialId: 349
Participant: 9601 | TrialId: 356
Participant: 9601 | TrialId: 360
Participant: 9601 | TrialId: 364
Participant: 9601 | TrialId: 366
Participant: 9601 | TrialId: 368
Participant: 9601 | TrialId: 369
Participant: 365 | TrialId: 377
Participant: 365 | TrialId: 378
Participant: 365 | TrialId: 379
Participant: 365 | TrialId: 380
Participant: 3

## Create full density Matrix and save to Database

In [66]:
sql_instruction = """SELECT Row, Column, SUM(Value) AS Value FROM density_map_participants GROUP BY Row, Column;"""
cr.execute(sql_instruction)
density_matrix_db = cr.fetchall()

In [67]:
density_matrix = create_matrix(step = 4, xmin = xMin, xmax = xMax, zmin = zMin, zmax = zMax)

In [68]:
for row in range(density_matrix.shape[0]):
    for column in range(density_matrix.shape[1]):
        for datapoint in density_matrix_db:
            if datapoint[0] == row and datapoint[1] == column:
                #print("Row: " + str(row) + " | Column: " + str(column) + " | value: " + str(datapoint[2]))
                density_matrix[row][column] = datapoint[2]

In [69]:
# create a table that stores general graph information
sql_instruction = """
CREATE TABLE IF NOT EXISTS "matrix_information"
("Id" INTEGER NOT NULL UNIQUE,
"Matrix_Name" TEXT NOT NULL,
PRIMARY KEY ("Id" AUTOINCREMENT))
"""
cr.execute(sql_instruction)

In [70]:
# create a table that stores the complete density matrix
sql_instruction = """
CREATE TABLE IF NOT EXISTS "density_matrix"(
"Id" INTEGER NOT NULL UNIQUE,
"MatrixId" INTEGER NOT NULL,
"Row" NUMERIC,
"Column" NUMERIC,
"Value" NUMERIC,
PRIMARY KEY ("Id" AUTOINCREMENT)
FOREIGN KEY(MatrixId) REFERENCES matrix_information(Id)
);
"""
cr.execute(sql_instruction)

In [71]:
matrix_in_databank = False

In [72]:
if not matrix_in_databank:
    #set_up_DB()
    
    # add the graph to the general table
    sql_instruction = """
    INSERT INTO matrix_information (Matrix_Name)
    VALUES ("Density_Matrix")
    """
    cr.execute(sql_instruction)
    
    # get the id from the graph
    cr.execute("""SELECT Id FROM matrix_information WHERE Matrix_Name = 'Density_Matrix'""")
    matrixId = cr.fetchone()
    matrixId = matrixId[0]
    
    # get mask information & store in database
    for row in range(density_matrix.shape[0]): #print(i)
        for column in range(density_matrix.shape[1]):
            value = str((matrixId,row,column, density_matrix[row][column]))
            sql_instruction = f"""INSERT INTO density_matrix (MatrixId, Row, Column, Value) VALUES {value};"""
            cr.execute(sql_instruction)
    connection.commit()
connection.close()